In [1581]:
import pandas as pd
import plotly.express as plt
import plotly.graph_objects as go

In [1582]:
eustat = pd.read_csv('GVA.csv')

In [1583]:
eustat.head(50)

,Country,1975,1976,1977,1978,1979,1980,1981,1982,1983,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Belgium,:,:,:,:,:,:,:,:,:,...,"18,713.50","19,008.00","19,294.00","19,678.40","20,149.90","21,767.40","22,628.00","22,019.80","24,298.90","26,484.10"
1,Bulgaria,:,:,:,:,:,:,:,:,:,...,"1,725.50","1,651.30","1,726.50","1,646.60","1,866.10","2,034.60","2,366.80","2,614.30","2,338.40","2,636.40"
2,Czechia,:,:,:,:,:,:,:,:,:,...,"8,215.70","8,048.30","8,636.50","8,721.30","9,547.70","10,630.70","11,489.50","11,088.60","12,110.80","14,423.30"
3,Denmark,"2,045.30","2,390.60","2,522.10","2,602.50","2,518.70","2,790.80","2,564.20","2,627.70","2,775.10",...,"10,145.20","10,532.70","11,749.90","13,380.40","14,328.80","14,828.70","14,716.50","14,860.20","16,275.10","19,183.40"
4,Germany,:,:,:,:,:,:,:,:,:,...,"112,226.00","119,583.00","124,907.00","132,548.00","137,994.00","148,044.00","154,323.00","167,329.00","179,802.00","211,102.00"
5,Estonia,:,:,:,:,:,:,:,:,:,...,"1,163.60","1,079.10","1,098.50","1,229.50","1,379.70","1,561.80","1,669.80","1,651.80","1,845.80","2,198.50"
6,Ireland,:,:,:,:,:,:,:,:,:,...,"3,477.50","4,427.40","4,908.80","5,734.50","7,478.10","8,055.20","8,778.90","8,178.60","8,717.80","11,051.30"
7,Greece,:,:,:,:,:,:,:,:,:,...,"5,153.90","3,738.40","3,493.40","3,484.20","2,259.10","2,431.20","2,573.70","2,681.30","2,925.10","3,945.10"
8,Spain,:,:,:,:,:,:,:,:,:,...,"53,763.00","53,254.00","56,422.00","59,362.00","62,061.00","64,459.00","70,821.00","61,880.00","60,865.00","63,162.00"
9,France,"20,036.70","21,680.90","22,869.30","25,723.60","28,587.40","33,291.00","34,214.40","36,563.70","36,565.60",...,"111,927.00","110,116.00","107,884.00","108,362.00","113,137.00","117,386.00","124,114.00","108,963.00","125,454.00","129,235.00"


In [1584]:
# Melt the dataset by 'Countries'
melted_eustat = pd.melt(eustat, id_vars='Country', var_name='Year', value_name='Gross Value Added (€M)')

# Sort the dataset by 'Country' and 'Year' (optional)
melted_eustat = melted_eustat.sort_values(['Country', 'Year']).reset_index(drop=True)

# Print the resulting melted dataset
melted_eustat.head(50)

,Country,Year,Gross Value Added (€M)
0,Albania,1975,:
1,Albania,1976,:
2,Albania,1977,:
3,Albania,1978,:
4,Albania,1979,:
5,Albania,1980,:
6,Albania,1981,:
7,Albania,1982,:
8,Albania,1983,:
9,Albania,1984,:


### Why Gross Value Added for the construction sector?
Gross Value Added (GVA) for the construction sector measures the value created by the industry. It reflects the difference between the value of goods and services produced and the cost of inputs used in the construction process. GVA is a vital indicator of economic performance and productivity within the construction sector. It helps determine the sector's contribution to the overall economy and provides insights into its growth and health. By subtracting the cost of materials and services from the total value of output, GVA represents the value added by the construction sector through production activities such as wages, profits, and other factors.

In [1585]:
country_codes = pd.read_csv('country_codes.tsv', sep='\t')

In [1586]:
# Merge the two datasets by 'Country'
eustat = pd.merge(melted_eustat, country_codes, left_on='Country', right_on='Country', how='left')
eustat.head(50)


,Country,Year,Gross Value Added (€M),Alpha-2 code,Alpha-3 code,Numeric
0,Albania,1975,:,AL,ALB,8
1,Albania,1976,:,AL,ALB,8
2,Albania,1977,:,AL,ALB,8
3,Albania,1978,:,AL,ALB,8
4,Albania,1979,:,AL,ALB,8
5,Albania,1980,:,AL,ALB,8
6,Albania,1981,:,AL,ALB,8
7,Albania,1982,:,AL,ALB,8
8,Albania,1983,:,AL,ALB,8
9,Albania,1984,:,AL,ALB,8


In [1587]:
# Let's drop the Alpha-2 code and Numeric columns because we don't need them
eustat = eustat.drop(['Alpha-2 code', 'Numeric'], axis=1)
eustat.head(50)

,Country,Year,Gross Value Added (€M),Alpha-3 code
0,Albania,1975,:,ALB
1,Albania,1976,:,ALB
2,Albania,1977,:,ALB
3,Albania,1978,:,ALB
4,Albania,1979,:,ALB
5,Albania,1980,:,ALB
6,Albania,1981,:,ALB
7,Albania,1982,:,ALB
8,Albania,1983,:,ALB
9,Albania,1984,:,ALB


In [1588]:
import numpy as np

# Let's convert the : to NaN
eustat['Gross Value Added (€M)'] = eustat['Gross Value Added (€M)'].replace(':', np.nan)  # Use np.nan, not 'NaN'

eustat.head(50)


,Country,Year,Gross Value Added (€M),Alpha-3 code
0,Albania,1975,NaN,ALB
1,Albania,1976,NaN,ALB
2,Albania,1977,NaN,ALB
3,Albania,1978,NaN,ALB
4,Albania,1979,NaN,ALB
5,Albania,1980,NaN,ALB
6,Albania,1981,NaN,ALB
7,Albania,1982,NaN,ALB
8,Albania,1983,NaN,ALB
9,Albania,1984,NaN,ALB


In [1589]:
# Let's convert the Year column to datetime
eustat['Year'] = pd.to_datetime(eustat['Year'])

# Let's make the Year column format to YYYY
eustat['Year'] = eustat['Year'].dt.year

eustat.head(50)



,Country,Year,Gross Value Added (€M),Alpha-3 code
0,Albania,1975,NaN,ALB
1,Albania,1976,NaN,ALB
2,Albania,1977,NaN,ALB
3,Albania,1978,NaN,ALB
4,Albania,1979,NaN,ALB
5,Albania,1980,NaN,ALB
6,Albania,1981,NaN,ALB
7,Albania,1982,NaN,ALB
8,Albania,1983,NaN,ALB
9,Albania,1984,NaN,ALB


In [1590]:
# Change the name of the column of "Alpha-3 code" to "Country Code"
eustat = eustat.rename(columns={'Alpha-3 code': 'Country Code'})

In [1591]:
eustat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1680 entries, 0 to 1679
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Country                 1680 non-null   object
 1   Year                    1680 non-null   int64 
 2   Gross Value Added (€M)  1046 non-null   object
 3   Country Code            1680 non-null   object
dtypes: int64(1), object(3)
memory usage: 65.6+ KB


In [1592]:
# Let's convert the GVA column to float type
eustat['Gross Value Added (€M)'] = eustat['Gross Value Added (€M)'].str.replace(',', '').astype(float)

eustat.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1680 entries, 0 to 1679
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Country                 1680 non-null   object 
 1   Year                    1680 non-null   int64  
 2   Gross Value Added (€M)  1046 non-null   float64
 3   Country Code            1680 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 65.6+ KB


In [1593]:
eustat.sample(5)

,Country,Year,Gross Value Added (€M),Country Code
783,Ireland,1990,NaN,IRL
1432,Slovakia,2015,5685.2,SVK
233,Bulgaria,2016,1646.6,BGR
1037,Malta,2004,343.9,MLT
765,Iceland,2020,1333.9,ISL


In [1594]:
# Let's make the GVA column format to millions
eustat['Gross Value Added (€M)'] = eustat['Gross Value Added (€M)'] * 1000000
eustat.sample(5)


,Country,Year,Gross Value Added (€M),Country Code
564,France,2011,1.116340e+11,FRA
947,Lithuania,2010,1.472600e+09,LTU
649,Greece,2000,8.948100e+09,GRC
678,Hungary,1981,NaN,HUN
1214,Poland,1989,NaN,POL


In [1595]:
# In the dataset GVA was in thousands, so we multiplied it by 1000000 to make it in millions. So let's change the name of the column to "Gross Value Added (€M)"
# We do not need to say "in millions" because we already multiplied it by 1000000
eustat = eustat.rename(columns={'Gross Value Added (€M)': 'Gross Value Added'})


In [1596]:
import plotly.express as px

# Calculate the minimum and maximum values of the 'Gross Value Added (€M)' column
min_value = eustat['Gross Value Added'].min()
max_value = eustat['Gross Value Added'].max()

# Create a choropleth map using the 'eustat' DataFrame, with countries represented by their country codes, and colors based on 'Gross Value Added (€M)'
fig = px.choropleth(
    eustat,
    locations='Country Code',
    hover_name='Country',
    color_continuous_scale='plasma',
    animation_frame='Year',
    color='Gross Value Added',
    title='GVA for Construction Sector',
    range_color=(min_value, max_value)
)

# Update the layout of the figure, including the title and the geographic projection
fig.update_layout(
    title_text='Gross Value Added for Construction Sector in Europe',
    geo=dict(
        scope='europe',
        projection_type='natural earth'
    ),
)

fig.show()


In [1597]:
# Let's see the highest GVA for each country
eustat.groupby('Country')['Gross Value Added'].max().sort_values(ascending=False).head(20)

Country
Germany           2.111020e+11
United Kingdom    1.513309e+11
France            1.292350e+11
Spain             1.155490e+11
Italy             8.965540e+10
Netherlands       4.010000e+10
Switzerland       3.670080e+10
Poland            3.625910e+10
Austria           3.137210e+10
Sweden            3.105260e+10
Belgium           2.648410e+10
Norway            2.444430e+10
Denmark           1.918340e+10
Romania           1.816360e+10
Greece            1.808580e+10
Ireland           1.736500e+10
Finland           1.587600e+10
Czechia           1.442330e+10
Portugal          1.052880e+10
Hungary           9.256500e+09
Name: Gross Value Added, dtype: float64

### Finding the reason of Grass Value Addition from Construction Sector in Ireland

Firstly, we will prepare the datasets for further analysis

# Number of Planning Permission

In [1598]:
planning_permission = pd.read_csv('Planning Permission Granted.csv')

In [1599]:
planning_permission.tail(50)

,Year,Total Floor Area for which Permission Granted(All types of Construction)
0,1977,NaN
1,1978,NaN
2,1979,NaN
3,1980,NaN
4,1981,NaN
5,1982,NaN
6,1983,NaN
7,1984,NaN
8,1985,NaN
9,1986,NaN


In [1600]:
# Let's make interpolation for the missing values
planning_permission = planning_permission.interpolate()

planning_permission.head(50)

,Year,Total Floor Area for which Permission Granted(All types of Construction)
0,1977,NaN
1,1978,NaN
2,1979,NaN
3,1980,NaN
4,1981,NaN
5,1982,NaN
6,1983,NaN
7,1984,NaN
8,1985,NaN
9,1986,NaN


In [1601]:
planning_permission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 2 columns):
 #   Column                                                                    Non-Null Count  Dtype  
---  ------                                                                    --------------  -----  
 0   Year                                                                      46 non-null     int64  
 1   Total Floor Area for which Permission Granted(All types of Construction)  31 non-null     float64
dtypes: float64(1), int64(1)
memory usage: 864.0 bytes


In [1602]:
# Let's fill the missing values with 0 to make the column type integer
planning_permission['Total Floor Area for which Permission Granted(All types of Construction)'].fillna(0, inplace=True) # Let's fill the missing values with 0

In [1603]:
# Let's change the Total Floor Area column to int type
planning_permission['Total Floor Area for which Permission Granted(All types of Construction)'] = planning_permission['Total Floor Area for which Permission Granted(All types of Construction)'].astype(int)

In [1604]:
# Let's convert the Year column to datetime and make the format YYYY before converting it to period
planning_permission['Year'] = pd.to_datetime(planning_permission['Year'], format='%Y').dt.to_period('Y') 

In [1605]:
# Let's set the Year column as index column
planning_permission.set_index('Year', inplace=True) 


In [1606]:
planning_permission.info()

<class 'pandas.core.frame.DataFrame'>
PeriodIndex: 46 entries, 1977 to 2022
Freq: A-DEC
Data columns (total 1 columns):
 #   Column                                                                    Non-Null Count  Dtype
---  ------                                                                    --------------  -----
 0   Total Floor Area for which Permission Granted(All types of Construction)  46 non-null     int64
dtypes: int64(1)
memory usage: 736.0 bytes


In [1607]:
planning_permission.tail(60)

,Total Floor Area for which Permission Granted(All types of Construction)
Year,
1977,0
1978,0
1979,0
1980,0
1981,0
1982,0
1983,0
1984,0
1985,0


In [1608]:
# Resample the data into quarterly frequency and fill the missing values with NaN
planning_permission = planning_permission.resample('Q').asfreq()

In [1609]:
# Interpolate the missing values within each quarter using linear interpolation method 
planning_permission.interpolate(method='linear', inplace=True)

In [1610]:
planning_permission.head(60)

,Total Floor Area for which Permission Granted(All types of Construction)
Year,
1977Q1,0.00
1977Q2,0.00
1977Q3,0.00
1977Q4,0.00
1978Q1,0.00
1978Q2,0.00
1978Q3,0.00
1978Q4,0.00
1979Q1,0.00


In [1611]:
# Let's change the name of Total Floor Area for which Permission Granted(All types of Construction) column to Construction Permission
planning_permission = planning_permission.rename(columns={'Total Floor Area for which Permission Granted(All types of Construction)': 'Construction Permission'})
planning_permission.sample(10)

,Construction Permission
Year,
2019Q2,8183.00
1993Q1,4614.00
1986Q4,0.00
1988Q3,0.00
1983Q2,0.00
1981Q2,0.00
1999Q2,12714.75
1979Q4,0.00
1980Q2,0.00


# New and Second Hand House Prices in Ireland

In [1612]:
house_prices = pd.read_csv('New House Prices in Ireland.csv')

In [1613]:
house_prices.tail(10)

,Quarter,New House Prices in IE (€),Second Hand House Prices(€)
158,2014Q3,247398,264394.0
159,2014Q4,258989,255958.0
160,2015Q1,267517,259385.0
161,2015Q2,275235,266787.0
162,2015Q3,285015,267530.0
163,2015Q4,298551,263616.0
164,2016Q1,309703,264258.0
165,2016Q2,314950,277452.0
166,2016Q3,304307,276759.0
167,2016Q4,322680,284348.0


In [1614]:
house_prices.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Data columns (total 3 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Quarter                      168 non-null    object 
 1   New House Prices in IE (€)   168 non-null    int64  
 2   Second Hand House Prices(€)  156 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 4.1+ KB


In [1615]:
# Let's change the name of Second Hand House Prices column to Second Hand House Prices in IE(€)
house_prices = house_prices.rename(columns={'Second Hand House Prices(€)': 'Second Hand House Prices in IE(€)'})

In [1616]:
# Let's convert the New House Prices column to int type
house_prices['New House Prices in IE (€)'] = house_prices['New House Prices in IE (€)'].astype(int)

In [1617]:
# replace non-finite values with 0
house_prices['Second Hand House Prices in IE(€)'].fillna(0, inplace=True)

In [1618]:
# Let's convert the Second-Hand House Prices column to int type
house_prices['Second Hand House Prices in IE(€)'] = house_prices['Second Hand House Prices in IE(€)'].astype(int)

In [1619]:
house_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Data columns (total 3 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   Quarter                            168 non-null    object
 1   New House Prices in IE (€)         168 non-null    int64 
 2   Second Hand House Prices in IE(€)  168 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 4.1+ KB


# Persons aged 15 years and over in Employment in Construction of Buildings(ILO)

In [1620]:
labour_force = pd.read_csv('Persons aged 15 years and over in Employment in Construction of Buildings(ILO).csv')

In [1621]:
labour_force.head(10)

,Quarter,Persons aged 15 years and over in Employment in Construction of Buildings (ILO)(Thousands)
0,1998Q1,45.1
1,1998Q2,46.4
2,1998Q3,50.6
3,1998Q4,50.9
4,1999Q1,52.9
5,1999Q2,54.4
6,1999Q3,57.4
7,1999Q4,57.9
8,2000Q1,59.9
9,2000Q2,63.5


In [1622]:
# Rename the column
labour_force = labour_force.rename(columns={'Persons aged 15 years and over in Employment in Construction of Buildings (ILO)(Thousands)': 'Labour Force'})


In [1623]:
labour_force.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Quarter       100 non-null    object 
 1   Labour Force  100 non-null    float64
dtypes: float64(1), object(1)
memory usage: 1.7+ KB


In [1624]:
# Let's multiply the Labour Force column by 1000 to make it in thousands
labour_force['Labour Force'] = labour_force['Labour Force'] * 1000

In [1625]:
labour_force.head(10)

,Quarter,Labour Force
0,1998Q1,45100.0
1,1998Q2,46400.0
2,1998Q3,50600.0
3,1998Q4,50900.0
4,1999Q1,52900.0
5,1999Q2,54400.0
6,1999Q3,57400.0
7,1999Q4,57900.0
8,2000Q1,59900.0
9,2000Q2,63500.0


In [1626]:
# Let's change Labour Force column to int type
labour_force['Labour Force'] = labour_force['Labour Force'].astype(int)